# Segmenting and Clustering Neighborhoods in Toronto

**Author**: Cody M. Jones  
**Date**: 30 June 2020

In [1]:
import numpy as np
import pandas as pd
import folium
import geocoder
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

## Data Wrangling

There's a data set on Wikipedia that contains all the information we need regarding postal codes in Toronto and their associated borough and neighborhood(s). This is a simple HTML table so we can use the `pd.read_html()` method to read the table straight into a data frame.

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfs = pd.read_html(io=url)
neighborhoods = dfs[0]
neighborhoods.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Let's drop all postal codes without an assigned borough for the sake of our analysis.

In [3]:
neighborhoods.drop(neighborhoods[neighborhoods['Borough'] == 'Not assigned'].index, inplace=True)
neighborhoods.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Check that all remaining postal codes have an assigned neighborhood.

In [4]:
len(neighborhoods[neighborhoods['Neighborhood'] == 'Not assigned'])

0

Now that we've cleaned up the data set, let's see how many postal codes are left for us to deal with.

In [5]:
print('Number of postal codes: {}'.format(neighborhoods.shape[0]))

Number of postal codes: 103


## Geocoding

In [6]:
coordinates = pd.read_csv('https://cocl.us/Geospatial_data')
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [7]:
neighborhoods = pd.merge(neighborhoods, coordinates, on='Postal Code')
neighborhoods.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## Clustering

Use the OpenStreetMap geocoder to find the coordinates of downtown Toronto.

In [8]:
g = geocoder.osm('Toronto, Canada')
latitude, longitude = g.latlng

Create an empty map centered on downtown Toronto.

In [9]:
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)

Add markers to the map for each postal code. 

In [10]:
# add markers to map
for latitude, longitude, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{} ({})'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.Marker([latitude, longitude], popup=label, parse_html=False).add_to(toronto_map)

Display the map with markers.

In [11]:
toronto_map

In [12]:
with open("credentials.txt", "r") as f:
    lines = [line.rstrip() for line in f.readlines()]

CLIENT_ID = lines[0]
CLIENT_SECRET = lines[1]
VERSION = lines[2]
LIMIT = 100

In [13]:
def get_venues(names, latitudes, longitudes, radius=500):
    venues = []

    for name, latitude, longitude in zip(names, latitudes, longitudes):
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, latitude, longitude, radius, LIMIT)
        
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues.append([(
            name, 
            latitude, 
            longitude, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    df = pd.DataFrame([item for venues in venues for item in venues])
    df.columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    
    return df

Call this function on each neighborhood and populate a new data frame with the venues returned by the Foursquare API.

In [14]:
venues = get_venues(names=neighborhoods['Neighborhood'], latitudes=neighborhoods['Latitude'], longitudes=neighborhoods['Longitude'])

Let's see the total number of venues returned by the API.

In [15]:
venues.shape[0]

2132

And break down the number of venues found in each neighborhood.

In [16]:
venues.groupby('Neighborhood').count()['Venue']

Neighborhood
Agincourt                                           5
Alderwood, Long Branch                              9
Bathurst Manor, Wilson Heights, Downsview North    21
Bayview Village                                     4
Bedford Park, Lawrence Manor East                  23
                                                   ..
Willowdale, Willowdale East                        35
Willowdale, Willowdale West                         7
Woburn                                              4
Woodbine Heights                                    7
York Mills West                                     2
Name: Venue, Length: 95, dtype: int64

In [17]:
print('There are {} unique categories.'.format(len(venues['Venue Category'].unique())))

There are 270 unique categories.


<a id='item3'></a>

### Analyze Each Neighborhood

We're going to perform a one-hot encoding on the venues data frame and prepend the neighborhood column in its string representation.

In [18]:
encoded = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

encoded.drop(labels='Neighborhood', axis='columns', inplace=True)
column = venues['Neighborhood']
encoded.insert(0, column.name, column)

encoded.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [19]:
encoded.shape

(2132, 270)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [20]:
grouped = encoded.groupby('Neighborhood').mean().reset_index()
grouped.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.043478,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Let's confirm the new size

In [21]:
grouped.shape

(95, 270)

Let's print out the first 5 neighborhoods along with their top 5 most common venue type.

In [22]:
num_top_venues = 5

for neighborhood in grouped['Neighborhood'][0:5]:
    print('Neighborhood(s): {}'.format(neighborhood))
    temp = grouped[grouped['Neighborhood'] == neighborhood].T.reset_index()
    temp.columns = ['Venue Type', 'Freq']
    temp = temp.iloc[1:]
    temp['Freq'] = temp['Freq'].astype(float)
    temp = temp.round({'Freq': 2})
    print(temp.sort_values('Freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print()

Neighborhood(s): Agincourt
                  Venue Type  Freq
0  Latin American Restaurant   0.2
1             Breakfast Spot   0.2
2               Skating Rink   0.2
3                     Lounge   0.2
4             Clothing Store   0.2

Neighborhood(s): Alderwood, Long Branch
       Venue Type  Freq
0     Pizza Place  0.22
1     Coffee Shop  0.11
2    Skating Rink  0.11
3  Sandwich Place  0.11
4             Pub  0.11

Neighborhood(s): Bathurst Manor, Wilson Heights, Downsview North
          Venue Type  Freq
0               Bank  0.10
1        Coffee Shop  0.10
2        Bridal Shop  0.05
3  Mobile Phone Shop  0.05
4        Supermarket  0.05

Neighborhood(s): Bayview Village
            Venue Type  Freq
0  Japanese Restaurant  0.25
1                 Bank  0.25
2   Chinese Restaurant  0.25
3                 Café  0.25
4    Accessories Store  0.00

Neighborhood(s): Bedford Park, Lawrence Manor East
           Venue Type  Freq
0         Coffee Shop  0.09
1      Sandwich Place  0.09
2     

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [23]:
def get_common_venues(row, num_top_venues):
    categories = row.iloc[1:]
    categories_sorted = categories.sort_values(ascending=False)
    return categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [24]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
sorted_ = pd.DataFrame(columns=columns)
sorted_['Neighborhood'] = grouped['Neighborhood']

for ind in np.arange(grouped.shape[0]):
    sorted_.iloc[ind, 1:] = get_common_venues(grouped.iloc[ind, :], num_top_venues)

sorted_.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Skating Rink,Breakfast Spot,Latin American Restaurant,Clothing Store,Yoga Studio,Drugstore,Distribution Center,Dog Run,Doner Restaurant
1,"Alderwood, Long Branch",Pizza Place,Pharmacy,Skating Rink,Dance Studio,Coffee Shop,Pub,Sandwich Place,Gym,American Restaurant,Colombian Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Chinese Restaurant,Supermarket,Deli / Bodega,Sushi Restaurant,Mobile Phone Shop,Shopping Mall,Restaurant,Ice Cream Shop
3,Bayview Village,Café,Japanese Restaurant,Bank,Chinese Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Yoga Studio
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Italian Restaurant,Restaurant,Sandwich Place,Sushi Restaurant,Grocery Store,Indian Restaurant,Juice Bar,Liquor Store,Comfort Food Restaurant


<a id='item4'></a>

### k-Means Clustering

Cluster the neighborhoods of Toronto with the k-means clustering algorithm.

In [25]:
# Number of clusters
k = 5

# Train the model
k_means = KMeans(n_clusters=k, random_state=0).fit(grouped.drop('Neighborhood', axis='columns'))

# Check out the cluster labels generated for each row in the data frame
k_means.labels_

array([0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 4, 3, 0, 0, 0, 0,
       4, 0, 0, 0, 0, 4, 1, 0, 1, 0, 1, 0, 0, 0, 3, 0, 4, 1, 0, 0, 0, 1,
       0, 0, 0, 4, 1, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 4,
       1, 0, 0, 4, 0, 0, 1], dtype=int32)

In [26]:
# Add cluster labels to the data frame
sorted_.insert(loc=0, column='Cluster Label', value=k_means.labels_)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [27]:
# Merged grouped with neighborhoods to add latitude/longitude for each neighborhood
merged = neighborhoods
merged = merged.join(sorted_.set_index('Neighborhood'), on='Neighborhood')

# Drop NaN rows and cast float to int in the 'Cluster Label' column
merged.dropna(axis='index', inplace=True)
merged['Cluster Label'] = merged['Cluster Label'].astype(int)

merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1,Park,Food & Drink Shop,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Electronics Store
1,M4A,North York,Victoria Village,43.725882,-79.315572,0,Portuguese Restaurant,Coffee Shop,French Restaurant,Hockey Arena,Intersection,Pizza Place,Electronics Store,Ethiopian Restaurant,Eastern European Restaurant,Drugstore
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Café,Theater,Beer Store,Spa,Shoe Store
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0,Clothing Store,Furniture / Home Store,Vietnamese Restaurant,Miscellaneous Shop,Shoe Store,Coffee Shop,Accessories Store,Event Space,Boutique,Electronics Store
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Diner,Sushi Restaurant,Yoga Studio,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Restaurant


### Cluster Visualization

In [28]:
# Create an empty map centered on Toronto
clusters_map = folium.Map(location=[latitude, longitude], zoom_start=11)

In [29]:
# Set the colorscheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [30]:
# Add markers to the map
markers_colors = []
for latitude, lonclusters_mapum.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(clusters_map)

SyntaxError: invalid syntax (<ipython-input-30-095b6a9fbce8>, line 3)

<a id='item5'></a>

In [ ]:
clusters_map

### Examine Clusters

#### First Cluster

In [ ]:
merged.loc[merged['Cluster Label'] == 0, merged.columns[[1] + list(range(5, merged.shape[1]))]]

### Second Cluster

In [ ]:
merged.loc[merged['Cluster Label'] == 1, merged.columns[[1] + list(range(5, merged.shape[1]))]]

### Third Cluster

In [ ]:
merged.loc[merged['Cluster Label'] == 2, merged.columns[[1] + list(range(5, merged.shape[1]))]]

### Fourth Cluster

In [ ]:
merged.loc[merged['Cluster Label'] == 3, merged.columns[[1] + list(range(5, merged.shape[1]))]]

### Fifth Cluster

In [ ]:
merged.loc[merged['Cluster Label'] == 4, merged.columns[[1] + list(range(5, merged.shape[1]))]]